In [ ]:
import sys

sys.path.append("..")
%load_ext autoreload
%autoreload 2
from trl_wrapper.trainer_wrapper import TrainerWrapper, GRPO_WRITING_CONFIG, SMOL_LM_135M, TXT_BT_DPO_CONFIG

cfg = TXT_BT_DPO_CONFIG
cfg.train_batch_size = 2
cfg.num_generations = 2
cfg.model_id_or_path = SMOL_LM_135M
cfg.notebook_mode = True
cfg.max_samples = 10

In [44]:
from dataset.writing import _sum_scores, _pick_completions
import pandas as pd
dataset_pd = pd.read_parquet("../dataset_files/backtranslate_best_of_n.parquet")

dataset_pd["instruction_id"] = (
    dataset_pd["instruction"].astype("category").cat.codes
)

dataset_pd["score_total"] = dataset_pd["scores"].apply(_sum_scores)

dataset_pd = dataset_pd.groupby("instruction_id").filter(
    lambda group: group["score_total"].max() != group["score_total"].min()
)
print(dataset_pd.columns)
dataset_pd = (
    dataset_pd.groupby("instruction_id").apply(_pick_completions).reset_index()
)

Index(['scores', 'completion', 'instruction', 'model_id', 'prompt_id',
       'instruction_id', 'score_total'],
      dtype='object')


/var/folders/h0/6v9c2xns5lbc32tsmk83rh000000gn/T/ipykernel_12117/4268088263.py:16: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dataset_pd.groupby("instruction_id").apply(_pick_completions).reset_index()


In [ ]:
wrapper = TrainerWrapper(cfg)
wrapper.init_model()

In [55]:
wrapper.init_data_module()

2025-04-01 22:28:52.436 | INFO     | trl_wrapper.trainer_wrapper:init_data_module:351 - Using chat template override: smollmv2
2025-04-01 22:28:52.437 | INFO     | trl_wrapper.wrapper_config:__init__:133 - Cache dir: ../dataset_caches/writing_d_p_o_data_module


Index(['scores', 'completion', 'instruction', 'model_id', 'prompt_id',
       'instruction_id', 'score_total'],
      dtype='object')


In [ ]:
wrapper.data_module.train_dataset

In [56]:
wrapper.init_trainer()

2025-04-01 22:28:59.993 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:388 - Saving output to: ./runs/04-01-22-28-940771-smollm2-135m-instruct--txt-bt-dpo
2025-04-01 22:28:59.995 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:411 - Initializing trainer, run_name: 04-01-22-28-940771-smollm2-135m-instruct--txt-bt-dpo, wandb project: gutenberg
2025-04-01 22:28:59.995 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:418 - logprobs cache location: ../dataset_caches/writing_d_p_o_data_module/b68b303b/ref_logprobs_cache peft config: False
Tokenizing eval dataset: 100%|██████████| 1034/1034 [00:02<00:00, 513.38 examples/s]
2025-04-01 22:29:22.895 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:637 - Precomputing reference logprobs, batch size: 16
2025-04-01 22:29:22.896 | INFO     | trl_wrapper.trainer_wrapper:init_trainer:644 - Precomputing train logprobs
Train dataset reference log probs:   0%|          | 0/582 [00:00<?, ?it/s]

: 

In [ ]:
from model.reasoning import GSM8K_SYSTEM_PROMPT, CONNECTIONS_PROMPT
from model.utils import get_available_device

messages = [
    {"role": "system", "content": CONNECTIONS_PROMPT},
    {
        "role": "user",
        "content": "train, panda, dove, series, wind, bear, orca, bass, string, skunk, speed, sand, zebra, tourist, desert, chain",
    },
]
tokenized_chat = wrapper.tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
    return_attention_mask=True,
)

device = get_available_device()
tokenized_chat = tokenized_chat.to(device)
out = wrapper.model.generate(tokenized_chat, max_length=1024)
print(wrapper.tokenizer.decode(out[0], skip_special_tokens=True))

In [ ]:
wrapper.train()